<a href="https://colab.research.google.com/github/Daryna-Ruban/Machine-learning/blob/main/%D0%A0%D1%83%D0%B1%D0%B0%D0%BD_%D0%9B%D0%917_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторна робота 7.
# Завдання 1

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357270 sha256=4770a09a6341c2412397ba65303f56f1e17a83ef5a8f24ed17029a8b9bad6b35
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [4]:
from surprise import Dataset
from surprise import Reader

# Завантажуємо MovieLens 100k вбудований у Surprise датасет
data = Dataset.load_builtin('ml-100k')

# Підготуємо дані для тренування
trainset = data.build_full_trainset()

# Перевіримо кількість користувачів та фільмів
print(f"Кількість користувачів: {trainset.n_users}")
print(f"Кількість фільмів: {trainset.n_items}")

Кількість користувачів: 943
Кількість фільмів: 1682


2. Виведіть перші 5 рядків завантаженого датасету.

In [5]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.
5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).

In [6]:
# Розділяємо дані на тренувальний і тестовий набори (80% тренування, 20% тестування)
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для підбору через GridSearchCV для SVD
param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

# Параметри для підбору через GridSearchCV для KNNBasic
param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

# Підбір параметрів для SVD
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

# Підбір параметрів для KNNBasic
grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [7]:
# Отримання найкращих моделей за MAE
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

# Виведення найкращих значень MAE для обох алгоритмів
print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

# Тренуємо обрану модель на тренувальному наборі
best_model.fit(trainset)

# Тестуємо модель на тестовому наборі
predictions = best_model.test(testset)

# Оцінюємо точність моделі
print("Оцінка точності моделі (RMSE):")
accuracy.rmse(predictions)

print("Оцінка точності моделі (MAE):")
accuracy.mae(predictions)

Найкращий MAE для SVD: 0.724806231909643
Найкращий MAE для KNNBasic: 0.8027616437934709
Вибраний алгоритм: SVD
Оцінка точності моделі (RMSE):
RMSE: 0.9215
Оцінка точності моделі (MAE):
MAE:  0.7241


0.7240764245565466

In [8]:
# Виведення рекомендацій для конкретного користувача
user_id = str(196)  # ID користувача для якого ми хочемо отримати рекомендації
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання всіх фільмів та тих, які користувач ще не рецензував
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 29
Топ-10 фільмів, рекомендованих для користувача:
Фільм 357 з прогнозованим рейтингом 4.59
Фільм 479 з прогнозованим рейтингом 4.54
Фільм 23 з прогнозованим рейтингом 4.51
Фільм 511 з прогнозованим рейтингом 4.50
Фільм 480 з прогнозованим рейтингом 4.48
Фільм 483 з прогнозованим рейтингом 4.46
Фільм 408 з прогнозованим рейтингом 4.46
Фільм 474 з прогнозованим рейтингом 4.46
Фільм 493 з прогнозованим рейтингом 4.46
Фільм 527 з прогнозованим рейтингом 4.45


# Завдання 2
1. Побудуйте власну рекомендаційну систему (приклад є в документації).
Оцініть її.
2. Отримайте рекомендацію для певного користувача

In [10]:
from surprise import Dataset

data = Dataset.load_builtin('jester')

trainset = data.build_full_trainset()

print(f"Кількість користувачів: {trainset.n_users}")
print(f"Кількість жартів: {trainset.n_items}")

Кількість користувачів: 59132
Кількість жартів: 140


In [11]:
print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

Перші 5 рядків датасету:
('1', '5', 0.219, None)
('1', '7', -9.281, None)
('1', '8', -9.281, None)
('1', '13', -6.781, None)
('1', '15', 0.875, None)


In [12]:
# Розділяємо на тренувальний та тестовий набори
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для пошуку гіперпараметрів SVD
param_grid_svd = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.1]
}

# Параметри для пошуку гіперпараметрів KNNBasic
param_grid_knn = {
    'k': [20, 30, 40],
    'min_k': [1, 3, 5],
    'sim_options': {
        'name': ['cosine', 'msd'],
        'user_based': [False]
    }
}

# Пошук найкращих моделей для SVD
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=3)
grid_search_svd.fit(data)

# Пошук найкращих моделей для KNNBasic
grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['mae'], cv=3)
grid_search_knn.fit(data)

# Отримання найкращих моделей за MAE
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

# Виведення найкращих значень MAE для обох алгоритмів
print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

# Тренуємо обрану модель на тренувальному наборі
best_model.fit(trainset)

# Тестуємо модель на тестовому наборі
predictions = best_model.test(testset)

# Оцінюємо точність моделі
print("Оцінка точності моделі (RMSE):")
accuracy.rmse(predictions)

print("Оцінка точності моделі (MAE):")
accuracy.mae(predictions)


KeyboardInterrupt: 

In [13]:
# Виведення рекомендацій для конкретного користувача
user_id = str(196)  # ID користувача для якого ми хочемо отримати рекомендації
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання всіх жартів та тих, які користувач ще не рецензував
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих жартів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих жартів
print("Топ-10 жартів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Жарт {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 22
Топ-10 жартів, рекомендованих для користувача:
Жарт 23 з прогнозованим рейтингом 4.51
Жарт 64 з прогнозованим рейтингом 4.45
Жарт 135 з прогнозованим рейтингом 4.43
Жарт 134 з прогнозованим рейтингом 4.35
Жарт 22 з прогнозованим рейтингом 4.29
Жарт 127 з прогнозованим рейтингом 4.22
Жарт 59 з прогнозованим рейтингом 4.21
Жарт 100 з прогнозованим рейтингом 4.18
Жарт 98 з прогнозованим рейтингом 4.13
Жарт 136 з прогнозованим рейтингом 4.13


Висновки: Модель SVD є однією з найкращих для рекомендаційних систем і показує досить високі прогнози, виходячи з індивідуальних вподобань користувача.

Користувачу з ID 196 рекомендовано кілька фільмів, які, за прогнозами моделі, можуть йому сподобатися найбільше, з передбаченими рейтингами вище 4.6, також за другим завданням обрано рекомендаційну систему, що підбирає 10 найкращих жартів. Це дозволяє очікувати, що система рекомендацій працює ефективно.

Аналіз результатів демонструє, що модель правильно прогнозує на основі поведінкових даних і забезпечує персоналізований підхід до користувача.